In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hermanus,ZA,-34.4187,19.2345,57.63,74,0,13.47,clear sky
1,1,Cape Town,ZA,-33.9258,18.4232,60.89,81,75,19.57,broken clouds
2,2,Terrasini,IT,38.1460,13.0844,84.22,62,0,16.11,clear sky
3,3,Baturaja,ID,-4.1333,104.1667,75.20,91,100,2.17,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,47.48,98,100,3.13,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Terrasini,IT,38.1460,13.0844,84.22,62,0,16.11,clear sky
3,3,Baturaja,ID,-4.1333,104.1667,75.20,91,100,2.17,overcast clouds
8,8,Bengkulu,ID,-3.8004,102.2655,75.29,90,87,5.14,overcast clouds
9,9,Bani,DO,18.2833,-70.3333,86.86,52,12,14.25,few clouds
15,15,Ribeira Grande,PT,38.5167,-28.7000,75.60,69,75,5.75,broken clouds
16,16,Kavieng,PG,-2.5744,150.7967,80.83,79,57,17.34,light rain
17,17,Puerto Ayora,EC,-0.7393,-90.3518,78.76,93,77,8.99,broken clouds
18,18,Sur,OM,22.5667,59.5289,82.51,72,100,16.26,overcast clouds
19,19,Georgetown,MY,5.4112,100.3354,80.89,85,20,4.61,few clouds
20,20,Faanui,PF,-16.4833,-151.7500,79.39,71,1,24.65,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                212
City                   212
Country                212
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [6]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Terrasini,IT,84.22,clear sky,38.1460,13.0844,
3,Baturaja,ID,75.20,overcast clouds,-4.1333,104.1667,
8,Bengkulu,ID,75.29,overcast clouds,-3.8004,102.2655,
9,Bani,DO,86.86,few clouds,18.2833,-70.3333,
15,Ribeira Grande,PT,75.60,broken clouds,38.5167,-28.7000,
16,Kavieng,PG,80.83,light rain,-2.5744,150.7967,
17,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
18,Sur,OM,82.51,overcast clouds,22.5667,59.5289,
19,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
20,Faanui,PF,79.39,light rain,-16.4833,-151.7500,


In [7]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):

            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Terrasini,IT,84.22,clear sky,38.1460,13.0844,Florio Park Hotel
3,Baturaja,ID,75.20,overcast clouds,-4.1333,104.1667,"HOTEL Sekar Sunja, Baturaja Timur, Ogan Komeri..."
8,Bengkulu,ID,75.29,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
9,Bani,DO,86.86,few clouds,18.2833,-70.3333,Hospedaje Henriquez
15,Ribeira Grande,PT,75.60,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
16,Kavieng,PG,80.83,light rain,-2.5744,150.7967,Nusa Island Retreat
17,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Sur,OM,82.51,overcast clouds,22.5667,59.5289,Sur Plaza Hotel
19,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
20,Faanui,PF,79.39,light rain,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   212
Country                212
Max Temp               212
Current Description    212
Lat                    212
Lng                    212
Hotel Name             212
dtype: int64

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "HotelDf.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [11]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description}
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))